# CUSTOMER CHURN CHALLENGE ON ZINDI

# Project Description

This challenge is for an African telecommunications company that provides customers with airtime and mobile data bundles. The objective of this challenge is to develop a machine learning model to predict the likelihood of each customer “churning”, i.e. becoming inactive and not making any transactions for 90 days.

This solution will help this telecom company to better serve their customers by understanding which customers are at risk of leaving.

# Hypothesis

Null Hypothesis: There is no relationship between the tenure and the churn of customers.

Alternate Hypothesis: There is a relationship between the tenure and the churn of customers.

# Analytical Questions

1. What is the overall churn rate of the telecommunication company?
2. What is the churn rate across the various regions?
3. What is the churn rate of custmers based on customer regularity?
4. What is the churn rate of customers based on their tenure?
(Group customers to short-term, mid-term and long term based on their tenure, then check the churn rate)
5. What is the churn rate of customers based on the top_up amounts?
(Group top_up amount to high, medium and low, then check the churn rate across the groups)
6. What is the churn rate of top pack users?
7. What is the churn rate based on the data volume?
8. What is the churn rate of custmers based on their income frequency?
9. What is the churn rate of customers based on their monthly income?
(Group monthly income to high-income earners, medium income earners, and low income earners, then check the churn rate across them)
10. What is the relationship between the monthly income of customers and the frequency of their recharge (the number of times a customer activated the top pack packages)?
11. What is the relationship between the number of times a customer made an income and the number of times a customer activated the top pack?
12. What is the churn distribution of customers based on calls to Zone1 and Zone 2?
13. What is the churn distribution of customers based international calls, calls to Orange, and calls to Tigo?

In [1]:
# Installations

!pip install huggingface_hub

In [2]:
# Import needed libraries
import pandas as pd
import numpy as np

# Library for working with operating system
import os

# Library for working with Google Drive
from google.colab import drive

# Libraries for visualization
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sb

# Library for testing the hypothesis
import scipy.stats as stats

# Libraries for features engineering
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Library for splitting the train data
from sklearn.model_selection import train_test_split

# Libraries for balancing the dataset
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Libraries for modelling
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

# Libraries for model evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve, f1_score, roc_auc_score, confusion_matrix

# Library for hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# Library for saving Machine Learning models
import pickle

# Library for pushing model to Huggingface
from huggingface_hub import notebook_login

# Library to handle warnings
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Set the visualization style

sb.set_style('darkgrid')
rcParams['figure.figsize'] = 6,4
%matplotlib inline

# Data Exploration

In [ ]:
# Load the datasets

# train = pd.read_csv('Train.csv')
# test=  pd.read_csv('Test.csv')
# submission_sample = pd.read_csv('SampleSubmission.csv')

In [ ]:
# Mount Google Drive

drive.mount('/content/drive')

In [ ]:
# Define the directory path to the datasets
data_path = '/content/drive/MyDrive/Google Colab/Customer Churn Challenge on Zindi/Data/'

# Define the directory path to save images
image_path = '/content/drive/MyDrive/Google Colab/Customer Churn Challenge on Zindi/Images'

# Define the directory path to save the "submission_file"
submission_path = '/content/drive/MyDrive/Google Colab/Customer Churn Challenge on Zindi/Submission/'

# Define the directory path to save exports
export_path = '/content/drive/MyDrive/Google Colab/Customer Churn Challenge on Zindi/Export/'

In [ ]:
# Load the datasets from the drive using the data_path

train = pd.read_csv(data_path + 'Train.csv')
test=  pd.read_csv(data_path + 'Test.csv')
submission_sample = pd.read_csv(data_path + 'SampleSubmission.csv')

In [ ]:
# Display the first five rows of the train dataset

train.head()

In [ ]:
# Display the last five rows of the train dataset

train.tail()

In [ ]:
# Obtain the shape of the train data

train.shape

In [ ]:
# View columns of the train data and their datatypes

train.info()

In [ ]:
# Display the first five rows of the test dataset

test.head()

In [ ]:
# Display the last five rows of the test dataset

test.tail()

In [ ]:
# Obtain the shape of the test dataset

test.shape

In [ ]:
# View the columns and datatypes of the test dataset

test.info()

In [ ]:
# List all the columns in the train and test dataset

train.columns, test.columns

The train and test datatypes have the same columns with the same datatypes, except that the train dataset has a 'CHURN' column which is absent in the test data. This 'CHURN' column is the target variable.

In [ ]:
# Display the first five rows of the submission_sample dataset

submission_sample.head()

In [ ]:
# Obtain the shape of the submission_sample dataset

submission_sample.shape

# Visualization of some columns in the *dataset*

In [ ]:
# Check the levels in the 'REGION' column

print(train['REGION'].value_counts())
plt.figure(figsize=(10,5))
train['REGION'].value_counts(normalize=True).plot(kind='bar')
plt.title('Levels in the REGION column')
plt.ylabel('counts')
plt.xlabel('REGION')

From the plot above, most of the customers are located in Dakar, followed by Thies and Saint-Louis. Kedougou is the region with the least number of customers.

In [ ]:
# Check the levels in the 'TENURE' column

print(train['TENURE'].value_counts())
plt.figure(figsize=(10,5))
train['TENURE'].value_counts(normalize=True).plot(kind='bar')
plt.title('Levels in the TENURE column')
plt.ylabel('counts')
plt.xlabel('TENURE')

A huge percentage of the customers captured in the train dataset have stayed with the telecommunication company for over 24 months. This implies that the dataset is focused on old customers.

In [ ]:
# Check the levels in the TENURE column of test dataset

print(test['TENURE'].value_counts())
plt.figure(figsize=(10,5))
test['TENURE'].value_counts(normalize=True).plot(kind='bar')
plt.title('Levels in the TENURE column of test dataset')
plt.ylabel('counts')
plt.xlabel('TENURE')

Both the train and test datasets focus on old customers, that is, customers that have been with the telecommunication network for over 24 months.

In [ ]:
# Check the levels in the MRG column

print(train['MRG'].value_counts())
plt.figure(figsize=(10,5))
train['MRG'].value_counts(normalize=True).plot(kind='bar')
plt.title('Levels in the MRG column')
plt.ylabel('counts')
plt.xlabel('MRG')

The MRG indicates if a client is going. The chart above shows that none of the customers in the train dataset is going.

In [ ]:
# Check the levels in the MRG column in the test dataset

print(test['MRG'].value_counts())
plt.figure(figsize=(10,5))
test['MRG'].value_counts(normalize=True).plot(kind='bar')
plt.title('Levels in the MRG column in the test dataset')
plt.ylabel('counts')
plt.xlabel('MRG')

The MRG indicates if a client is going. The chart above shows that none of the customers in the test dataset is going.

In [ ]:
# Check the top 40 levels in the TOP_PACK column

print(train['TOP_PACK'].value_counts().head(40))
plt.figure(figsize=(10,5))
train['TOP_PACK'].value_counts(normalize=True).head(40).plot(kind='bar')
plt.title('Top 40 Levels in the TOP_PACK column')
plt.ylabel('counts')
plt.xlabel('TOP_PACK')

In [ ]:
# Check if the predictor class is balanced

print(train['CHURN'].value_counts())
plt.figure(figsize=(10,5))
train['CHURN'].value_counts(normalize=True).plot(kind='bar')
plt.title('Levels in the predictor class')
plt.ylabel('counts')
plt.xlabel('CHURN')

In [ ]:
# Check the summary of numerical fields and apply transpose to easily display all the columns

summary_statistics = train.select_dtypes(include=['int64', 'float64']).describe().T
summary_statistics

In [ ]:
# Plot the correlation matrix using heatmap

sb.heatmap(summary_statistics.corr(), annot=True)

In [ ]:
# Check for missing values in train dataset

train.isnull().sum()

In [ ]:
# Check for missing values in test dataset

test.isnull().sum()

# Hypothesis Testing

Null Hypothesis: There is no relationship between the tenure and the churn of customers.

Alternate Hypothesis: There is a relationship between the tenure and the churn of customers.

In [ ]:
# Define the null hypothesis and alternative hypothesis
null_hypothesis = 'There is no relationship between the tenure and churn of customers.'
alternative_hypothesis = 'There is a relationship between the tenure and churn of customers.'

# Perform the chi-square test
observed = pd.crosstab(train['TENURE'], train['CHURN'])
chi2, p_value, _, _ = stats.chi2_contingency(observed)

# Set the significance level
alpha = 0.05

# Print the test results
print('Null Hypothesis:', null_hypothesis)
print('Alternative Hypothesis:', alternative_hypothesis)
print('Significance Level (alpha):', alpha)
print('Chi-square statistic:', chi2)
print('P-value:', p_value)

# Compare the p-value with the significance level
if p_value < alpha:

    print(f'Result: Reject the null hypothesis.', alternative_hypothesis)
else:
    print(f'Result: Fail to reject the null hypothesis.', null_hypothesis)

 The P-value of 0.0 suggests a strong evidence against the null hypothesis, indicating a statistically significant result. There is a strong relationship between customer tenure and customer churn.

# Answering Analytical Questions

## Questions

1. What is the overall churn rate of the telecommunication company?
2. What is the churn rate across the various regions?
3. What is the churn rate of custmers based on customer regularity?
4. What is the churn rate of customers based on their tenure?
(Group customers to short-term, mid-term and long term based on their tenure, then check the churn rate)
5. What is the churn rate of customers based on the top_up amounts?
(Group top_up amount to high, medium and low, then check the churn rate across the groups)
6. What is the churn rate of top pack users?
7. What is the churn rate based on the data volume?
8. What is the churn rate of custmers based on their income frequency?
9. What is the churn rate of customers based on their monthly income?
(Group monthly income to high-income earners, medium income earners, and low income earners, then check the churn rate across them)
10. What is the relationship between the monthly income of customers and the frequency of their recharge (the number of times a customer activated the top pack packages)?
11. What is the relationship between the number of times a customer made an income and the number of times a customer activated the top pack?
12. What is the churn distribution of customers based on calls to Zone1 and Zone 2?
13. What is the churn distribution of customers based international calls, calls to Orange, and calls to Tigo?

Question 1: What is the overall churn rate of the telecommunication company?

In [ ]:
# Calculate the churn rate
total_customers = len(train)
churned_customers = train['CHURN'].sum()
churn_rate = (churned_customers / total_customers) * 100

# Display the churn rate
print('Total Customers:', total_customers)
print('Churned Customers:', churned_customers)
print(f'Churn Rate: {churn_rate.round(1)}%')

# Plot the churn rate
# plt.figure(figsize=(6, 4))
plt.bar(['Churned', 'Not Churned'], [churn_rate, 100-churn_rate])
plt.title('Overall Churn Rate Of The Telecommunication Network')
plt.xlabel('Churn Status')
plt.ylabel('Percentage')
plt.ylim([0, 100])

# Save the plot into the drive through the image_path
file_name = 'Overall Churn Rate Of The Telecommunication Network.png' # Specify the file name and extension
file_path = os.path.join(image_path, file_name) # Combine the file name with the image_path
plt.savefig(file_path)

# Display the chart
plt.show()

201,993 customers have churned out of 1,077,024 customers in the train dataset. This represents an 18.8% churn rate.

Question 2: What is the churn rate across the various regions?

In [ ]:
# Identify the regions

regions = train['REGION'].unique()
regions

Notice that the 'REGION' column above has missing values. The percentage of these rows with missing values will be evaluated.

In [ ]:
# Calculate the total number of rows with missing values in the 'REGION' column
missing_rows = train['REGION'].isnull().sum()

# Calculate the total number of rows in the DataFrame
total_rows = len(train)

# Calculate the percentage of rows with missing values in the 'REGION' column
percentage_missing = (missing_rows / total_rows) * 100

print(f"The percentage of rows with missing values in 'REGION' column is: {percentage_missing:.1f}%")

39.4 % of the rows in the 'REGION" column have missing values. These rows will be dropped in order to properly analyze the rest of the rows. A new copy of the train dataset will be created for this purpose.

In [ ]:
# Create a copy of the train dataset
train_df = train.copy()

# Drop rows with missing values in the 'REGION' column
train_df = train_df.dropna(subset=['REGION'])

# Identify the regions and the customers per region
regions = train_df['REGION'].unique()
customers_per_region = train_df['REGION'].value_counts()
percentage_of_customers_per_region = ((customers_per_region / customers_per_region.sum()) * 100).round(1)

# Initialize lists to store data for all regions
churned_customers_per_region = []
churn_rate_per_region = []

# Function to calculate the churn rate of different regions
for region in regions:
    region_df = train_df[train_df['REGION'] == region]
    churned_customers = region_df['CHURN'].sum()
    total_customers = len(region_df)

    if total_customers > 0:
        churn_rate = (churned_customers / total_customers) * 100
    else:
        churn_rate = 0.0

    churn_rate = churn_rate.round(1)

    churned_customers_per_region.append(churned_customers)
    churn_rate_per_region.append(churn_rate)

# Create a dictionary to store the region and churn data
churn_rate_by_region = {
    'REGION': regions,
    'CUSTOMERS PER REGION': customers_per_region,
    'PERCENTAGE OF CUSTOMERS': percentage_of_customers_per_region,
    'CHURNED CUSTOMERS PER REGION': churned_customers_per_region,
    'CHURN RATE (%)': churn_rate_per_region
}

# Display the region and churn data as a DataFrame
churn_rate_by_region_df = pd.DataFrame(churn_rate_by_region).reset_index(drop=True)
churn_rate_by_region_df

The DataFrame above shows the number of customers in each region, the percentage of customers in each region, the number of customers that churned, and the churn rate of each region. The highest churn rate is observed in Sedhiou Region with a churn rate of 5.41% and Kedougou region with a churn rate of 4.09%. Also observe that these regions have the lowest number (and percentage) of customers. This suggests that the telecommunication network should intensify customer retention policies in these regions. The region with a very low churn rate is Kaffrine region with a churn rate of 0.7%. Dakar region has the highest number (and percentage) of customers with a moderate churn rate.

In [ ]:
# Plot the churn rate per region
# plt.figure(figsize=(6, 4))
plt.bar(churn_rate_by_region_df.index, churn_rate_by_region_df['CHURN RATE (%)'], tick_label=churn_rate_by_region_df['REGION'])
plt.title('Churn Rate by Region')
plt.xlabel('Region')
plt.ylabel('Churn Rate (%)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Save the plot into the drive through the image_path
file_name = 'Churn Rate by Region.png'
file_path = os.path.join(image_path, file_name)
plt.savefig(file_path)

# Show the chart
plt.show()

As shown above, Sedhiou region has the highest churn rate followed by Kedougou region, while Kaffrine region has the lowest churn rate.

Question 3: What is the churn rate of customers based on customer regularity?

Regularity refers to the number of times a customer is active for 90 days. The most active and least active customers will be determined first, and checked for churn.

In [ ]:
# Find the highest and lowest values in the "REGULARITY" column
highest_regularity = train['REGULARITY'].max()
lowest_regularity = train['REGULARITY'].min()

# Find the user ID of the most active customer and least active customer
most_active = train[train['REGULARITY'] == highest_regularity]['user_id'].values[0]
least_active = train[train['REGULARITY'] == lowest_regularity]['user_id'].values[0]

# Check if the most active customer and least active customer churned
churned_most_active = train[train['user_id'] == most_active]['CHURN'].values[0]
churned_least_active = train[train['user_id'] == least_active]['CHURN'].values[0]

# Print the results
print(f"Customer {most_active} had the highest regularity of {highest_regularity} and {'churned' if churned_most_active == 1 else 'did not churn'}.")
print(f"Customer {least_active} had the lowest regularity of {lowest_regularity} and {'churned' if churned_least_active == 1 else 'did not churn'}.")

As shown above. the most active and least active customers did not churn.

Next, the usage activity of the customers will be used to group them into Low-Usage Customers, Medium-Usage Customers, and High-Usage Customers. These groups of customers will then be checked for churn.

In [ ]:
# Define the usage groups and corresponding labels
bins = [0, 20, 40, float('inf')]
labels = ['Low-Usage Customers', 'Medium-Usage Customers', 'High-Usage Customers']

# Create a new column 'usage_group' to store the usage group labels
train['usage_group'] = pd.cut(train['REGULARITY'], bins=bins, labels=labels)

# Calculate the churn rate, the number of customers and the percentage of customers in each group
churn_rate_by_group = train.groupby('usage_group')['CHURN'].mean()
customers_per_group = train['usage_group'].value_counts()
percentage_customers_per_group = round((customers_per_group / customers_per_group.sum()) * 100,1)

# Remove the 'usage_group' column
train.drop('usage_group', axis=1, inplace=True)

# Create a DataFrame to display the results
usage_group_data = pd.DataFrame({'Number of Customers': customers_per_group, 'Percentage of Customers in Group(%)': percentage_customers_per_group, 'Churn Rate (%)': churn_rate_by_group})

# Display the DataFrame
usage_group_data


As shown in the DataFrame above, there are more Low-Usage Customers (499,031), and this group of customers have the highest churn rate (0.379327). While High-Usage Customers (375,819) have the lowest churn rate (0.009004). This suggests that customer retention policies should be targeted at the Low-Usage Customers.

In [ ]:
# Plot the churn rate across the usage groups
# plt.figure(figsize=(6, 4))
churn_rate_by_group.plot(kind='bar', color='skyblue', alpha=0.7)
plt.title('Churn Rate Across Usage Groups Based On Regularity')
plt.xlabel('Usage Groups Based on Regularity')
plt.ylabel('Churn Rate')
plt.xticks(rotation=45)
plt.tight_layout()

# Save the plot into the drive through the image_path
file_name = 'Churn Rate Across Usage Groups Based On Regularity.png'
file_path = os.path.join(image_path, file_name)
plt.savefig(file_path)

# Show the chart
plt.show()

The chart above illustrates that Low-Usage Customers have the highest churn rate while High-Usage Customers have the lowest churn rate.

Question 4: What is the churn rate of customers based on their tenure?
A customer's tenure refers to the customer's duration in the network. The unique tenures are identified below.

In [ ]:
# Identify the unique tenures in the 'TENURE' group

train['TENURE'].unique()

The tenures are grouped into categories. The churn rate across these tenure groups will be examined.

In [ ]:
# Calculate the number of customers, the percentage of customers, and the churn rate for each tenure
customers_per_tenure = train['TENURE'].value_counts()
percentage_customers_per_tenure = ((customers_per_tenure / customers_per_tenure.sum()) * 100).round(2)
churn_rate_by_tenure = (train.groupby('TENURE')['CHURN'].mean()).round(2) # 2 decimal places where used in both to properly display the results

# Create a DataFrame to display the results
tenure_group_data = pd.DataFrame({
    'Number of Customers': customers_per_tenure,
    'Percentage of Customers (%)': percentage_customers_per_tenure,
    'Churn Rate (%)': churn_rate_by_tenure
})

# Display the DataFrame
tenure_group_data

As shown above, 94.88% of the customers have been with the network for 24 months and above (that is, 2 years and above). The churn rate among this group of customers is 0.18%, which is relatively low. The highest churn rate is 0.32% and it is observed for customers within 12-15 months duration. After one year with the network, these customers churned (became inactive for 90 days). Promotional activities such as one-year bonanza may help to retain these set of customers on the network.

In [ ]:
# Plot a bar chart for the churn rate by tenure
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 2)
tenure_group_data['Churn Rate (%)'].plot(kind='bar')
plt.title("Churn Rate by Tenure")
plt.xlabel("Tenure")
plt.ylabel("Churn Rate (%)")
plt.xticks(rotation=65)
plt.tight_layout()

# Save the plot into the drive through the image_path
file_name = 'Churn Rate by Tenure.png'
file_path = os.path.join(image_path, file_name)
plt.savefig(file_path)

# Display the chart
plt.show()

# Feature Engineering


Feature engineering is the process of creating new, meaningful, and informative features (variables) from the existing data to improve the performance of machine learning models. It involves creating, selecting, or transforming features that help models better understand patterns, relationships, and trends within the data.

Effective feature engineering can lead to more accurate and efficient models because the process would provide relevant data with reduced noise, which would enhance the predictive power of the data.

Feature engineering requires domain knowledge and creativity to extract valuable insights and knowledge from the raw data, which can significantly impact the success of machine learning projects.

The feature engineering techniques that will be used in this project are: feature selection, data sampling, feature extraction, data splitting, feature scaling and feature encoding. These techniques will enhance the modelling process.

Since changes will be made on the train and test datasets during the feature engineering processes, copies of these datasets will first be made for future usage.

In [ ]:
# Make copies of the train and test datasets before commencing the features engineering processes

train_data = train.copy()
test_data = test.copy()

## Feature Selection

This is a feature engineering process that is used to choose the desired columns (columns that are relevant to the modelling) for further processing. This means that some of the columns of the train dataset will be dropped at this point, leaving only the relevant columns.

In [ ]:
# List the columns in the train dataset

train.columns

In [ ]:
# Identify the columns to drop
columns_to_drop = ['user_id', 'REGION', 'ZONE1', 'ZONE2', 'MRG', 'TOP_PACK']

# Drop these columns in the train dataset using the "drop()" method
train.drop(columns=columns_to_drop, axis=1, inplace=True)
train.head()

In [ ]:
# Drop these columns in the test datast using the "drop()" method

test.drop(columns=columns_to_drop, axis=1, inplace=True)
test.head()

## Data Sampling

Large datasets can be computationally expensive and time-consuming to work with. Data sampling is a technique used to create a smaller, representative subset of the data for model development and testing. This can make the modeling process more efficient.

In [ ]:
# Check the shape of the train dataset

train.shape

Since the train dataset is very large (with 1,077,024 rows), a subset of the data would be used to train the models in order to adequately manage time and available computational resources.

There are many rows with missing values across the columns of the train dataset. This is as shown below.

In [ ]:
# identify the columns with missing values on the train dataset

train.isnull().sum()

In [ ]:
# Calculate the percentage of missing values for each column on the train dataset
train_missing_percentage = (train.isnull().mean() * 100).round(2)

# Create a DataFrame to display the results
train_missing_percentage_df = pd.DataFrame({'Missing Percentage (%)': train_missing_percentage})

# Display the DataFrame
train_missing_percentage_df

A great percentage of rows across the columns of the train dataset have missing values, filling them by imputation would introduce some bias to the dataset. Dropping these rows would be a better decision. This will also help to generate a subset of the train dataset which would optimize the modelling process.

'DATA_VOLUME', 'TIGO', 'ORANGE', and 'FREQ_TOP_PACK' are the columns with the highest percentage of missing values in the train dataset. The rows with missing values across these columns will be dropped to achieve manual data sampling.

In [ ]:
# Create a variable for the columns with high missing values

columns_with_high_missing_values = ['DATA_VOLUME', 'TIGO', 'ORANGE', 'FREQ_TOP_PACK']

In [ ]:
# Drop the rows with missing values across these columns in train dataset
train = train.dropna(subset=columns_with_high_missing_values)

# Verify that there are very little missing values
train.isnull().sum()

As can be seen above, there are now fewer rows with missing values across the columns of the train dataset.

In [ ]:
# Recheck the percentage of missing values for each column on the train dataset

train_missing_percentage = (train.isnull().mean() * 100).round(2)
train_missing_percentage_df = pd.DataFrame({'Missing Percentage (%)': train_missing_percentage})
train_missing_percentage_df

The percentage of missing values across the columns of the train dataset has greatly reduced.

In [ ]:
# Check the shape of the train dataset

train.shape

A reasonable subset of the train datset has been obtained which will help in managing available time, as well as computational resources for the modelling process.

The manual data sampling technique (dropping of some rows with missing values) achieved the following results:
- The percentage of rows with missing values reduced across the columns. This would ensure that filling the remaining few missing values by imputation will not introduce significant bias to the dataset.
- The number of rows in the train dataset reduced, giving a reasonable subset that is still suitable for the modelling process. This would ensure that available time and computational resources are well-managed.

**Next steps for handling missing values:**
- The remaining missing values in the train dataset will be handled by imputation.
- All the missing values in the test dataset will also be handled by imputation. Rows with missing values in the test dataset will not be dropped in order to maintain the number of rows in the dataset, as each row represents a seperate user whose churn prediction is expected.

**Note:** The imputation described above will be done after the feature extraction and data splitting processes to generate the training set and validation set. This means that the training and validation sets will be imputed seperately. Imputing them seperately will avoid the possiblility of data leakage, which can reduce the effectiveness of the validation set. The validation set is an important component for model evaluation.



## Feature Extraction

Feature extraction transforms data into a reduced set of relevant and informative features, which can be more meaningful and suitable for use in Data Analysis and Machine Learning tasks, making it easier for Machine Learning algorithms to learn and make accurate predictions or classifications.

The feature extraction technique used in this project aims to obtain the features (X) and target (y) variables of the train dataset, before splitting the dataset to obtain the training set and validation set.

In [ ]:
# Obtain the features (X) and target (y) variables on the train dataset
y = train['CHURN']
X = train.drop(columns=['CHURN'], axis=1)

# Print the shape of the features
print(X.shape)
print(y.shape)
print(test.shape)

## Data Splitting

In [ ]:
# Split the features(X) and target(y) variables to obtain the training and validation sets
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.5,random_state=42)

# Print the shape of the training and validation sets
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

## Imputation

Imputation is the process of filling in missing values in a dataset with estimated or calculated values. It is typically done to ensure that the dataset is complete and can be used for various data analysis and machine learning tasks. Common imputation strategies include filling missing values with the mean, median, mode, or other statistical measures.

Since the columns with missing values are numeric columns and the percentage of missing values are now relatively small, a SimpleImputer with strategy set to 'mean' will be used to fill the missing values. The training set, validation set, and test dataset will be fitted to the imputer and transformed seperately to avoid data leakage.

In [ ]:
# Initialize the imputer

imputer = SimpleImputer(strategy='mean')

In [ ]:
# Specify all the numeric columns

num_cols = ['MONTANT', 'FREQUENCE_RECH', 'REVENUE', 'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO',
            'REGULARITY', 'FREQ_TOP_PACK']

In [ ]:
# Fit and transform the numerical columns of the training set
X_train[num_cols] = imputer.fit_transform(X_train[num_cols])

# Verify that the training set has been transformed by the imputer
X_train.isnull().sum()

In [ ]:
# Fit and transform the numerical columns of the validation set
X_val[num_cols] = imputer.fit_transform(X_val[num_cols])

# Verify that the validation set has been transformed by the imputer
X_val.isnull().sum()

In [ ]:
# Fit and transform the numerical columns of the test dataset
test[num_cols] = imputer.fit_transform(test[num_cols])

# Verify that the test dataset has been transformed by the imputer
test.isnull().sum()

## Feature Scaling

Scaling is the process of transforming numerical features in a dataset to have a common scale or range. It is important when the numerical features have different units or magnitudes, as some machine learning algorithms are sensitive to feature scales. StandardScaler (z-score normalization) and Min-MaxScaler are common methods to scale features to a similar range.

For this project, StandardScaler will be used because of it's robustness and compatibility with a wide range of Machine Learning models. The training set, validation set, and test dataset will be fitted to the scaler and transformed seperately to avoid data leakage.

In [ ]:
# Initialize the scaler

scaler = StandardScaler()

In [ ]:
# Fit and transform the numerical columns of training set
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])

# Verify that the numerical columns of training set have been transformed by the scaler
X_train.head()

In [ ]:
# Fit and transform the numerical columns of validation set
X_val[num_cols] = scaler.fit_transform(X_val[num_cols])

# Verify that the numerical columns of validation set have been transformed by the scaler
X_val.head()

In [ ]:
# Fit and transform the numerical columns of test data
test[num_cols] = scaler.fit_transform(test[num_cols])

# Verify that the numerical columns of test data have been transformed by the scaler
test.head()

## Feature Encoding

Encoding is the process of converting categorical (non-numeric) variables into a numerical format, which is suitable for machine learning algorithms. Categorical variables cannot be used directly by most Machine Learning models, so they need to be encoded into numerical representations.
OneHotEncoder, LabelEncoder, and BinaryEncoder are commonly used for encoding categorical variables.

Since the dataset for this project is considerably large, LabelEncoder will be used for its simplicity and memory-efficiency. The training set, validation set, and test dataset will be fitted to the encoder and transformed seperately to avoid data leakage.

Note that the only categorical column on the datasets is the 'TENURE' column.

In [ ]:
# Initialize the encoder

encoder = LabelEncoder()

In [ ]:
# Fit and transform the 'TENURE' column of training set
X_train['TENURE'] = encoder.fit_transform(X_train['TENURE'])

# Verify that the 'TENURE' column of training set has been transformed by the encoder
X_train.head()

In [ ]:
# Fit and transform the 'TENURE' column of validation set
X_val['TENURE'] = encoder.fit_transform(X_val['TENURE'])

# Verify that the 'TENURE' column of validation set has been transformed by the encoder
X_val.head()

In [ ]:
# Fit and transform the 'TENURE' column of test data
test['TENURE'] = encoder.fit_transform(test['TENURE'])

# Verify that the 'TENURE' column of test data has been transformed by the encoder
test.head()

## Class Balancing

Class balancing aims to address the issue of class imbalance, where one class has significantly fewer samples compared to the other class. Techniques for class balancing include oversampling the minority class, undersampling the majority class, or a combination of both to create a balanced training set. Balancing the classes is essential to avoid model bias and ensure that the Machine Learning models can effectively learn from the available data.

For this project, there is an imbalance in the predictor class (target variable) of the training set. This imbalance will be handled by oversampling the minority class and undersampling the majority class as shown below. The Synthetic Minority Over-sampling Technique (SMOTE) and the RandomUnderSampler will be used respectively to achieve this. These techniques are preferred as this project involves binary classification.

In [ ]:
# Perform oversampling using SMOTE
smote = SMOTE(random_state=42)
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

# Perform undersampling using RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_train_balanced, y_train_balanced = rus.fit_resample(X_train_oversampled, y_train_oversampled)

# Print the class distribution before and after balancing
print("Before balancing:")
print(y_train.value_counts())
print("\nAfter balancing:")
print(pd.Series(y_train_balanced).value_counts())

# Modelling and Evaluation

Modelling refers to the process of selecting and training a mathematical or computational representation of a real-world phenomenon. This representation is designed to capture the essential relationships, patterns, and behaviors observed in data. The primary goal of modeling is to make predictions, gain insights, or simulate the behavior of the real-world system, which may be too complex or difficult to analyze directly. There are different models for Machine Learning processes. The choice of modeling technique and the specific steps involved can vary greatly depending on the problem domain and the available data. The ultimate goal of modeling is to generate data-driven insights and use them to solve problems or make informed decisions.

The models to be used for this project are:
- Logistic Regression
- Random Forest
- Gradient Boosting
- Adaptive Boosting
- Support Vector Machine
- Gaussian Naive Bayes

These models will be trained on the imbalanced training set, and the balanced training set to determine which set is giving better results. The results of each training process is measured by evaluating the performance of the models on the respective validation sets with the use of performance metrics.

Performance metrics are the measures or criteria used to evaluate the performance/effectiveness of a machine learning model. The performance of the models above will be evaluated with the following performance metrics:
- Accuracy: Accuracy measures the overall correctness of a model's predictions by calculating the ratio of correctly predicted instances (both true positives and true negatives) to the total number of instances in the dataset. It is useful when all classes have similar importance and the class distribution is balanced. It can be misleading in imbalanced datasets.
- Precision: Precision assesses the ability of a model to correctly identify positive instances among all instances it predicts as positive. It is the ratio of true positives to the total number of instances predicted as positive. Precision is important when the cost of false positives (Type I errors) is high, and you want to minimize the rate of false alarms.
- Recall (Sensitivity or True Positive Rate): Recall evaluates the model's ability to capture all positive instances in the dataset. It is the ratio of true positives to the total number of actual positive instances. Recall is crucial when the cost of false negatives (Type II errors) is high, and you want to minimize the rate of missed positive cases.
- F1 Score: The F1 score is a balanced metric that combines Precision and Recall into a single value. It is the harmonic mean of Precision and Recall. A higher F1 score indicates a better balance between Precision and Recall. F1 score is useful when you want a single metric that considers both false positives and false negatives. It's particularly relevant when class distribution is imbalanced.
- Area Under the Curve (AUC): AUC measures the ability of a model to discriminate between positive and negative instances across different probability thresholds. It is often used in the context of receiver operating characteristic (ROC) curves. A higher AUC indicates a better ability to rank positive instances higher than negative instances. AUC is valuable for evaluating the overall quality of a binary classifier. It's especially useful when the dataset may have class imbalance or when you need to compare the performance of different models.

**The primary evaluation metric is Area Under the Curve (AUC).**

The models will be tuned to optimize their performance, then trained and evaluated again to observe if the performance metrics improved. The results of the performance metrics for each training process will be stored in a DataFrame for easy comparison.

In [ ]:
# Create a list of models to train and evaluate

models = [
    ('Logistic Regression', LogisticRegression(random_state=42, solver='lbfgs')),
    ('Random Forest', RandomForestClassifier(bootstrap=True, criterion="gini", n_jobs=-1)),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42, n_estimators=50, subsample=1.0)),
    ('Adaptive Boosting', AdaBoostClassifier(random_state=42, n_estimators=50, learning_rate=1.0)),
    ('Support Vector Machine', SVC(random_state=42, C=1.0)),
    ('Gaussian Naive Bayes', GaussianNB())
]

## Model training and evaluation with imbalanced dataset

In [ ]:
# Initialize a dictionary to store the performance metrics of the models after training with imbalanced dataset
imbal_performance_metrics = {}

# Model training, evaluation and result calculation
for model_name, model in models:
    # Model training with imbalanced dataset
    model.fit(X_train, y_train)

    # Using the models to make predictions on the validation set
    y_pred_imbal = model.predict(X_val)

    # Calculate the performance metrics of the models on the imbalanced dataset
    accuracy = accuracy_score(y_val, y_pred_imbal)
    precision = precision_score(y_val, y_pred_imbal)
    recall = recall_score(y_val, y_pred_imbal)
    f1 = f1_score(y_val, y_pred_imbal)
    auc = roc_auc_score(y_val, y_pred_imbal)

    # Store the performance metrics results
    imbal_performance_metrics[model_name] = {
        'Imbal Accuracy': accuracy,
        'Imbal Precision': precision,
        'Imbal Recall': recall,
        'Imbal F1 Score': f1,
        'Imbal AUC': auc
    }

In [ ]:
# Create a DataFrame to store the performance metrics of the models on the imbalanced dataset
imbalanced_performance_metrics = pd.DataFrame(imbal_performance_metrics).transpose()

# Arrange the performance metrics DataFrame in descending order based on the AUC score
imbalanced_performance_metrics = imbalanced_performance_metrics.sort_values('Imbal AUC', ascending=False)

# Show the performance metrics DataFrame of the models on the imbalanced dataset
imbalanced_performance_metrics.style.set_caption('The Performance Metrics Of The Models On The Imbalanced Dataset')

From the Performance Metric DataFrame above, the best performing model on the imbalanced train dataset is the Gaussian Naive Bayes with an AUC score of 0.722989. Observe that this model also has the highest Recall score (0.711884) and the second highest F1 score (0.120612), but the lowest Accuracy score (0.733524) and one of the lowest Precision scores (0.065888). Here are the implications of these observations:

- High AUC Score: A high AUC score (0.722989) indicates that the Gaussian Naive Bayes model performs well in terms of distinguishing between the two classes (positive and negative). It means the model has a good ability to rank positive instances higher than negative ones.

- High Recall Score: The high Recall score (0.711884) suggests that the Gaussian Naive Bayes model has a low rate of false negatives. It correctly identifies a significant portion of the actual positive cases. In the context of binary classification, this is important when there is need to minimize the number of positive cases that are missed.

- Low Accuracy Score: The low Accuracy score (0.733524) indicates that the model doesn't perform well in terms of overall correct predictions. Accuracy alone is not the best metric for imbalanced datasets, especially when there is a significant class imbalance. In this case, the model's imbalance could be contributing to the low accuracy.

- Low Precision Score: The low Precision score (0.065888) suggests that the model has a high rate of false positives. It means that when the model predicts a positive case, it's often incorrect. This is reflected in the low precision score, which is calculated as the ratio of true positives to the total predicted positives.

- Second-Highest F1 Score: The second-highest F1 score (0.120612) indicates that the model achieves a balance between precision and recall. The F1 score is the harmonic mean of precision and recall, and it is often used when there is an imbalance between the classes. The Gaussian Naive Bayes model has a relatively good F1 score, although it's not the highest among the models.

## Model training and evaluation with balanced dataset

In [ ]:
# Initialize a dictionary to store the performance metrics of the models after training with balanced dataset
bal_performance_metrics = {}

# Model training, evaluation and result calculation
for model_name, model in models:
    # Model training with balanced dataset
    model.fit(X_train_balanced, y_train_balanced)

    # Using the models to make predictions on the validation set
    y_pred_bal = model.predict(X_val)

    # Calculate the performance metrics of the models on the balanced dataset
    accuracy = accuracy_score(y_val, y_pred_bal)
    precision = precision_score(y_val, y_pred_bal)
    recall = recall_score(y_val, y_pred_bal)
    f1 = f1_score(y_val, y_pred_bal)
    auc = roc_auc_score(y_val, y_pred_bal)

    # Store the performance metrics results
    bal_performance_metrics[model_name] = {
        'Bal Accuracy': accuracy,
        'Bal Precision': precision,
        'Bal Recall': recall,
        'Bal F1 Score': f1,
        'Bal AUC': auc
    }

In [ ]:
# Create a DataFrame to store the performance metrics of the models on the balanced dataset
balanced_performance_metrics = pd.DataFrame(bal_performance_metrics).transpose()

# Arrange the performance metrics DataFrame in descending order based on the AUC score
balanced_performance_metrics = balanced_performance_metrics.sort_values('Bal AUC', ascending=False)

# Show the performance metrics DataFrame of the models on the balanced dataset
balanced_performance_metrics.style.set_caption('The Performance Metrics Of The Models On The Balanced Dataset')

From the Performance Metric DataFrame above, the best performing model on the balanced train dataset based on AUC score is the Support Vector Machine as it has the highest AUC score (0.773081). While the Gaussian Naive Bayes that had the best AUC score on the imbalanced dataset (0.722989) now has the worst AUC score on the balanced dataset (0.628601). Also observe that the Support Vector Machine has the highest Accuracy score (0.783825), the highest Precision score (0.085163), the highest F1 score (0.153198), and a moderate Recall score (0.761755) on the balanced dataset. Here are the implications of these observations on Support Vector Machine (SVM):

High AUC Score (0.773081): A high AUC (Area Under the Curve) score indicates that the SVM model has a strong ability to distinguish between the positive and negative classes. This suggests that the model's predictions and the actual target values are well separated, which is a positive sign for the model's overall performance.

High Accuracy Score (0.783825): The high accuracy score indicates that the SVM model makes a high proportion of correct predictions on the balanced dataset. This suggests that the model is effective in classifying both positive and negative cases.

High Precision Score (0.085163): Precision measures the ratio of true positive predictions to all positive predictions made by the model. The high precision score indicates that when the SVM model predicts the positive class (that is, "1" meaning "CHURN"), it is accurate in doing so. In other words, it has a low rate of false positives.

High F1 Score (0.153198): The F1 score is the harmonic mean of precision and recall. A high F1 score indicates a balance between precision and recall. In this case, the SVM model achieves a good trade-off between identifying true positives and minimizing false positives.

Moderate Recall Score (0.761755): Recall measures the ratio of true positive predictions to all actual positive cases. While the SVM model's recall score is not the highest, it is still reasonably good. The model is effective in identifying most of the actual positive cases, although there is some room for improvement.

The performance of the models on the imbalanced and balanced datasets will be compared by concatenating the DataFrames.

In [ ]:
# Concatenate the performance metrics DataFrames for the imbalanced and balanced datasets while preserving columns
cocatenate_performance_metrics = pd.concat([imbalanced_performance_metrics, balanced_performance_metrics], axis=1)

# Arrange the combined performance metrics DataFrame in descending order based on the AUC score for the balanced dataset
cocatenate_performance_metrics = cocatenate_performance_metrics.sort_values('Bal AUC', ascending=False)

# Show the DataFrame
cocatenate_performance_metrics.style.set_caption('The Performance Metrics Of The Models On The Imbalanced And Balanced Datasets')

As can be seen above, there is significant improvement on the AUC score of each model on the balanced dataset in comparison to the imbalanced dataset, with an exception for Gaussian Naive Bayes whose AUC score dropped on the balanced dataset. Notwithstanding, the AUC Score of Support Vector Machine on the balanced dataset (0.773081) is still higher than that of Gaussian Naive Bayes on the imbalanced dataset (0.722989). These observations imply that the models generally performed better on the balanced dataset.

Since Support Vector Machine and Logistic Regression performed very well on the balanced dataset compared to the other models, with an AUC Score of 0.773081 and 0.769256 respectively, these two models will be passed through hyperparameter tuning. After performing hyperparameter tuning on these models, they will be trained again on the balanced dataset to see if their performance will improve.

# Hyperparameter tuning

Hyperparameters are adjustable parameters whose values control the model training process.

Hyperparameter tuning (or hyperparameter optimization) is a process used to determine the right combination of hyperparameters that maximizes the model performance. It works by running multiple trials on a model in a single training process. The parameters of the model are then set within specified limits and executed to identify the set of parameter values that give optimal results for the model.

Of all the models trained on the balanced dataset, Support Vector Machine and Logistic Regression are the best performing models. These two models will be passed through hyperparameter tuning.

In [ ]:
# Create a list of models to pass through hyperparameter tuning

hyperparameter_models = [
    ('Logistic Regression', LogisticRegression(random_state=42, solver='lbfgs')),
    ('Support Vector Machine', SVC(random_state=42, C=1.0))
]

# Get the available parameters for each model
for model_name, model in hyperparameter_models:
    available_params = model.get_params()
    print(f'Available Parameters For {model_name}:{available_params}\n')

In [ ]:
# Initialize a dictionary to store the tuned models
tuned_models = {}

# Initialize a dictionary to store the performance metrics of the tuned models
tun_performance_metrics = {}

# Perform hyperparameter tuning within specified limits
for model_name, model in hyperparameter_models:
    params_selection = {
        'Logistic Regression': {'solver': ['newton-cg', 'lbfgs', 'liblinear'], 'C': [100, 10, 1.0]},
        'Support Vector Machine': {'kernel': ['poly', 'rbf', 'sigmoid'], 'C': [100, 10, 1.0]}
    }

    # Get the selected parameter values for the models to tune
    param_grid = params_selection[model_name]

    # Perform hyperparameter tuning using GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc', verbose=0, refit=True)
    grid_search.fit(X_train_balanced, y_train_balanced)

    # Get the tuned models with their best parameters
    best_params = grid_search.best_params_
    tuned_model = grid_search.best_estimator_

    # Store the tuned models
    tuned_models[model_name] = tuned_model

    # Show the best parameters for each tuned model
    print(f'The best parameters for {model_name} are {best_params}\n')

    # Training each tuned model and making predictions on the validation set
    tuned_model.fit(X_train_balanced, y_train_balanced)
    y_pred_tun = tuned_model.predict(X_val)

    # Calculate the performance metrics on the balanced dataset for each tuned model
    accuracy = accuracy_score(y_val, y_pred_tun)
    precision = precision_score(y_val, y_pred_tun)
    recall = recall_score(y_val, y_pred_tun)
    f1 = f1_score(y_val, y_pred_tun)
    auc = roc_auc_score(y_val, y_pred_tun)

    # Store the performance metrics results
    tun_performance_metrics[model_name] = {
        'Tuned Accuracy': accuracy,
        'Tuned Precision': precision,
        'Tuned Recall': recall,
        'Tuned F1 Score': f1,
        'Tuned AUC': auc
    }

In [ ]:
# Create a DataFrame to store the performance metrics of the tuned models on the balanced dataset
tuned_performance_metrics = pd.DataFrame(tun_performance_metrics).transpose()

# Arrange the performance metrics DataFrame in descending order based on the AUC score of the tuned models
tuned_performance_metrics = tuned_performance_metrics.sort_values('Tuned AUC', ascending=False)

# Show the performance metrics DataFrame of the tuned models on the balanced dataset
tuned_performance_metrics.style.set_caption('The Performance Metrics Of The Tuned Models On The Balanced Dataset')

## Obtain the best model

In [ ]:
# Initialize a dictionary to store the best models
best_models = {}

# Iterate through the models and their AUC scores
for model_name, metrics in tun_performance_metrics.items():
    auc_score = metrics['Tuned AUC']

    # Update the best model in the dictionary if the current model has a higher AUC score
    if not best_models or auc_score > best_auc_score:
        best_auc_score = auc_score
        best_model_name = model_name
        best_model = tuned_models[model_name]

# Display the best model_name with the highest AUC score
print(f'The best model is {best_model_name} with an AUC score of {best_auc_score}')

## Confusion matrix of the best model

In [ ]:
# Make predictions on the validation set
best_model_pred = best_model.predict(X_val)

# Compute the confusion matrix
confusion = confusion_matrix(y_val, best_model_pred)
confusion

# Plot the confusion matrix using a heatmap
sb.heatmap(confusion, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix of the Best Model')

# Save the plot into the drive through the image_path
file_name = 'Confusion Matrix of the Best Model.png'
file_path = os.path.join(image_path, file_name)
plt.savefig(file_path)

# Show the plot
plt.show()

## Precision-recall curve of the best model

In [ ]:
# Use the best model to predict probabilities
y_scores = best_model.predict_proba(X_val)[:, 1]

# Calculate precision and recall
precision, recall, thresholds = precision_recall_curve(y_val, y_scores)

# Print precision, recall, and thresholds
for p, r, t in zip(precision, recall, thresholds):
    print(f"Precision: {p:.2f} | Recall: {r:.2f} | Threshold: {t:.2f}")

# Plot the precision-recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='b', lw=2, label='Precision-Recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid()
plt.legend(loc='upper right')

# Save the plot into the drive through the image_path
file_name = 'Precision-Recall Curve of the Best Model.png'
file_path = os.path.join(image_path, file_name)
plt.savefig(file_path)

# Show the plot
plt.show()

# Prediction on test dataset

The best model will be used to make predictions on the test dataset in order to identify the customers at the risk of churning. This customer churn prediction on the test dataset will subsequently be used to fill the 'CHURN' column of the submission dataset.

In [ ]:
# Make predictions on the test dataset using the best model
test_pred = best_model.predict(test_data)

# Print the test dataset predictions
test_pred

Remember that copies of the train and test datasets named "train_data" and "test_data" respectively were made before commencing the feature engineering processes. These copies contain all the original columns of each dataset. The "test_data" dataset is important at this stage as it contains the original 'user_id' column of "test" dataset.

A 'CHURN' column will be created for the "test_data" dataset to store the churn prediction results of each customer for their respective user_ids.

In [ ]:
# Create a 'CHURN' column for the test_data dataset and fill it with the test dataset predictons

test_data['CHURN'] = test_pred
test_data

# Submission

The 'user_id' and 'CHURN' columns of the "test_data" dataset will be used to create a new dataset for submission named "submission_data"

The "submission_data" dataset will then be saved into the "Submission" folder on the Google Drive. The path to this folder on the drive has been defined already (immediately after the drive was mounted).

In [ ]:
# First display the the first five rows of "submission_sample" dataset

submission_sample.head()

In [ ]:
# Create the "submission_data" dataset
submission_data = test_data[['user_id', 'CHURN']]

# Display the first five rows of "submission_data" dataset
submission_data.head()

In [ ]:
# Save the "submission_data" as a CSV into the drive through the submission_path

file_name = 'submission_data.csv'
file_path = os.path.join(submission_path, file_name)
submission_file.to_csv(file_path, index=False) # Set index to False as there is no index on the "submission_sample" dataset

# Exportation

The key Machine Learning components which are the imputer, the scaler, the encoder, and the best model will be exported so that they can be re-used later. When exported, these Machine Learning components can be deployed into apps and APIs for wide-scale customer churn predictions.

The exportation process begins with saving the Machine Learning components into a dictionary. This dictionary will be used to store these components into a picke file named "ml_components.pkl", which will then be saved into the "Export" folder on the Google Drive. The path to this folder on the drive has been defined already (immediately after the drive was mounted).

Another file named "requirements.txt" will also be created. This file saves a list of installed Python packages and their versions used for this project. Creating this file is a convenient way to capture the virtual environment dependencies used to run this project, making it possible for these same virtual environment dependencies to be re-used in the future to make changes to this project, or run a similar project. The "requirements.txt" file will also be saved into the "Export" folder on the Google Drive.

In [ ]:
# Create a dictionary to store all the Machine Learning components
ml_components = {
    'imputer': imputer,
    'scaler': scaler,
    'encoder': encoder,
    'model': best_model
}

# Create a pickle file to store the Machine Learning components and save it into the drive
with open(os.path.join(export_path, 'ml_components.pkl'), 'wb') as file:
    pickle.dump(ml_components, file)

# Create a requirements.txt file to capture the virtual environment and also save it into the drive
os.system(f'pip freeze > {export_path}/requirements.txt')